In [ ]:
import serial
import time
import joblib
import re
from flask import Flask, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

# Load model
model = joblib.load("") # replace with the path where you have your trained pkl file

# Connect to Serial
ser = serial.Serial('COM10', 115200, timeout=1)
time.sleep(2)
print("✅ Serial connected to ESP32")

# Helper: Read lines for a fixed duration
def read_lines_for(duration=1.5):
    lines = []
    start_time = time.time()
    while time.time() - start_time < duration:
        try:
            line = ser.readline().decode('utf-8', errors='ignore').strip()
            if line:
                lines.append(line)
        except Exception as e:
            print("⚠ Serial read error:", e)
    return lines

# Extract temp + hum
def extract_temp_hum(lines):
    for line in lines:
        if "Temperature:" in line and "Humidity:" in line:
            try:
                match = re.search(r'Temperature:\s*([\d.]+)°C\s*\|\s*Humidity:\s*([\d.]+)%', line)
                if match:
                    return float(match.group(1)), float(match.group(2))
            except Exception as e:
                print("❌ TEMP parse error:", e)
    return None, None

# Extract RFID (Improved & Stable)
def extract_rfid(lines):
    rfid_map = {
        "93ECB030": "FreshApple",
        "F3108AF5": "FreshBanana"
    }

    for line in lines:
        if "UID" in line or "RFID" in line or "card detected" in line:
            if "No RFID card detected" in line:
                continue  # Skip no card lines
            match = re.search(r'UID:\s*([0-9A-Fa-f\s]+)', line)
            if match:
                raw_uid = match.group(1)
                uid = raw_uid.replace(" ", "").upper()
                food_item = rfid_map.get(uid, "Unknown")
                return uid, food_item

    return None, "No Card"

# Route: Temperature only
@app.route('/temperature', methods=['GET'])
def get_temperature():
    try:
        lines = read_lines_for()
        temp, hum = extract_temp_hum(lines)
        if temp is not None and hum is not None:
            condition = model.predict([[temp, hum]])[0]
            condition_label = "✅ Safe" if condition == 0 else "⚠ Unsafe"
            return jsonify({
                "temperature": temp,
                "humidity": hum,
                "condition": condition_label
            }), 200
        return jsonify({"error": "No valid DHT data found"}), 500
    except Exception as e:
        print("❌ Error in /temperature:", e)
        return jsonify({"error": str(e)}), 500

# Route: RFID only (Stable now)
@app.route('/rfid', methods=['GET'])
def get_rfid():
    try:
        lines = read_lines_for()
        tag, status = extract_rfid(lines)
        return jsonify({
            "rfid_tag": tag,
            "rfid_status": status
        }), 200
    except Exception as e:
        print("❌ Error in /rfid:", e)
        return jsonify({"error": str(e)}), 500

# Route: Predict both
@app.route('/predict', methods=['GET'])
def predict_all():
    try:
        lines = read_lines_for(2)
        temp, hum = extract_temp_hum(lines)
        tag, status = extract_rfid(lines)

        if temp is None or hum is None:
            return jsonify({"error": "Temperature/humidity data missing"}), 500

        condition = model.predict([[temp, hum]])[0]
        condition_label = "✅ Safe" if condition == 0 else "⚠ Unsafe"

        return jsonify({
            "temperature": temp,
            "humidity": hum,
            "condition": condition_label,
            "rfid_tag": tag,
            "rfid_status": status
        }), 200
    except Exception as e:
        print("❌ Error in /predict:", e)
        return jsonify({"error": str(e)}), 500

# Run the app
app.run(host='0.0.0.0', port=5002)


c:\Users\venka\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\venka\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


✅ Serial connected to ESP32
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5002
 * Running on http://192.168.114.213:5002
Press CTRL+C to quit
127.0.0.1 - - [29/Apr/2025 21:58:55] "GET /temperature HTTP/1.1" 500 -
127.0.0.1 - - [29/Apr/2025 21:58:55] "GET /rfid HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2025 21:58:58] "GET /rfid HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2025 21:58:58] "GET /temperature HTTP/1.1" 500 -
127.0.0.1 - - [29/Apr/2025 21:59:01] "GET /rfid HTTP/1.1" 200 -
c:\Users\venka\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
127.0.0.1 - - [29/Apr/2025 21:59:02] "GET /temperature HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2025 21:59:03] "GET /rfid HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2025 21:59:07] "GET /rfid HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2025 21:59:07] "GET /temperature HTTP/1.1" 500 -
127.0.0.1 - - [29/Apr/2025 21:59:10] "GET /rfid HTTP/1.1" 2